### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Hypothyroidism/GSE75685'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check for gene expression data availability
is_gene_available = True  # Assume gene expression data is available based on background

# Check for variable availability and type for 'Hypothyroidism'
if 'personal pathological hystory: Hypothyroidism' in sample_characteristics[21]:
    trait_row = 21

# Check for variable availability and type for 'age'
if len(set(sample_characteristics[19])) > 1:  # ensure variability in age data
    age_row = 19

# Check for variable availability and type for 'gender'
if len(set(sample_characteristics[1])) > 1:  # ensure variability in gender data
    gender_row = 1

# Define data type conversion functions

def convert_trait(value):
    if value.endswith("Hypothyroidism"):
        return 1
    elif value.endswith("Neg"):
        return 0
    else:
        return None

def convert_age(value):
    try:
        return float(value.split(':')[-1].strip())
    except ValueError:
        return None

def convert_gender(value):
    gender = value.split(':')[-1].strip().lower()
    if gender == 'female':
        return 0
    elif gender == 'male':
        return 1
    else:
        return None

# Save cohort information
save_cohort_info('GSE75685', './preprocessed/Hypothyroidism/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Hypothyroidism', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Hypothyroidism/trait_data/GSE75685.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
